In [1]:
import torch
from decoder_net import ImageNet, RangeNet, DataSet
import matplotlib.pyplot as plt
import os
import numpy as np
import math
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()
print('Device: ', device)

net = RangeNet()

data = DataSet('warehouse')

# epoch_loss, batch_loss = net.train_net(net, data, device, 1, 1000)

net.load_state_dict(torch.load('trained_conv_nets/rng/warehouse1'))

Device:  cuda


<All keys matched successfully>

In [55]:
def plot_ranges(ranges):
    ranges = ranges.detach().numpy().flatten()
    ranges_x = []
    ranges_y = []
    for i, r in enumerate(ranges):
        ranges_x.append(r * math.cos(math.radians(i + 1)))
        ranges_y.append(r * math.sin(math.radians(i + 1)))

    return ranges_x, ranges_y

In [61]:
train_data_office_rng = data.X_test

ranges_pred = net(data.X_test)
ranges_true = data.Y_rng_test

ranges_pred_x, ranges_pred_y = plot_ranges(ranges_pred[200])
ranges_true_x, ranges_true_y = plot_ranges(ranges_true[200])
fig, axs = plt.subplots(2)
fig.set_figheight(10)

axs[0].scatter(ranges_pred_x, ranges_pred_y)
axs[1].scatter(ranges_true_x, ranges_true_y)
plt.show()

In [13]:
ranges_pred[200].detach()

tensor([[1.1464, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960,
         2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964,
         2.1960, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960, 2.1964, 2.1960,
         2.1964, 2.1960, 2.5002, 2.5013, 2.8495, 2.8489, 2.8002, 2.7994, 2.8423,
         2.8422, 2.6019, 2.5997, 2.1964, 2.1960, 3.0212, 3.0249, 3.9694, 3.9685,
         3.8355, 3.8342, 3.9498, 3.9503, 3.2973, 3.2919, 2.1964, 2.1960, 2.8835,
         2.8865, 3.6734, 3.6725, 3.5618, 3.5606, 3.6570, 3.6573, 3.3033, 3.2996,
         2.6044, 2.6039, 3.2440, 3.2467, 4.0410, 4.0404, 3.7819, 3.7797, 3.6215,
         3.6218, 4.0593, 4.0596, 4.2775, 4.2765, 4.1203, 4.1189, 4.2545, 4.2551,
         4.4865, 4.4851, 4.3415, 4.3405, 4.1795, 4.1780, 4.3178, 4.3184, 4.0947,
         4.0910, 3.4138, 3.4131, 3.9581, 3.9602, 4.7679, 4.7678, 4.2166, 4.2120,
         3.5488, 3.5491, 4.2783, 4.2801, 4.8464, 4.8453, 4.6462, 4.6445, 4.8171,
         4.8180, 5.0081, 5.0

In [28]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()
print('Device: ', device)

net = ImageNet()

data = DataSet('office')

epoch_loss, batch_loss = net.train_net_img(net, data, device, 2, 20)

Device:  cuda
Starting training using office data.


In [8]:
plt.plot(range(len(epoch_loss)), epoch_loss)
plt.title('Training of deconv net on office data')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

/home/simon/.local/lib/python3.8/site-packages/ipykernel/eventloops.py:106: UserWarning: Attempted to set non-positive left xlim on a log-scaled axis.
Invalid limit will be ignored.
  app.exec_()
